# Reduction of ESTIA McStas data

This notebook demonstrates how to run the data reduction on the output of McStas simulations of the instrument.

Essentially this looks very similar to how one would do data reduction on real data files from the physical instrument,
but we replace the default loader with the `load_mcstas_events` provider.

In [ ]:
%matplotlib widget
import scipp as sc

from ess.estia.load import load_mcstas_events
from ess.estia.data import estia_mcstas_example
from ess.estia import EstiaWorkflow

from ess.reflectometry.types import *

from ess.reflectometry.figures import wavelength_z_figure, wavelength_theta_figure, q_theta_figure


wf = EstiaWorkflow()
wf.insert(load_mcstas_events)
wf[Filename[ReferenceRun]] = estia_mcstas_example('reference')

wf[YIndexLimits]  = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(14 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')
wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 2001, unit='angstrom')
wf[QBins] = 200

# There is no proton current data in the McStas files, here we just add some fake proton current
# data to make the workflow run.
wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})



In [ ]:
results = {}
for path in estia_mcstas_example('Ni/Ti-multilayer'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    R, da = w.compute((ReflectivityOverQ, ReducibleData[SampleRun])).values()
    results[f"{da.coords['sample_rotation'].value} {da.coords['sample_rotation'].unit}"] = R.hist()

sc.plot(results, norm='log', vmin=1e-8)

In [ ]:
results = []
for path in estia_mcstas_example('Ni/Ti-multilayer'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    results.append(w.compute(ReducibleData[SampleRun]))

In [ ]:
wavelength_z_figure(results[3], wavelength_bins=400)

In [ ]:
wavelength_theta_figure(results, wavelength_bins=400, theta_bins=200, q_edges_to_display=[sc.scalar(0.016, unit='1/angstrom'), sc.scalar(0.19, unit='1/angstrom')])

In [ ]:
q_theta_figure(results, q_bins=300, theta_bins=200)

In [ ]:
results = {}
for path in estia_mcstas_example('Ni-film on silicon'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    R, da = w.compute((ReflectivityOverQ, ReducibleData[SampleRun])).values()
    results[f"{da.coords['sample_rotation'].value} {da.coords['sample_rotation'].unit}"] = R.hist()

sc.plot(results, norm='log', vmin=1e-9)

In [ ]:
results = []
for path in estia_mcstas_example('Ni-film on silicon'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    results.append(w.compute(ReducibleData[SampleRun]))

In [ ]:
wavelength_z_figure(results[3], wavelength_bins=400)

In [ ]:
wavelength_theta_figure(results, wavelength_bins=400, theta_bins=200, q_edges_to_display=[sc.scalar(0.016, unit='1/angstrom'), sc.scalar(0.19, unit='1/angstrom')])

In [ ]:
q_theta_figure(results, q_bins=300, theta_bins=200)

In [ ]:
results = {}
for path in estia_mcstas_example('Natural SiO2 on silicon'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    R, da = w.compute((ReflectivityOverQ, ReducibleData[SampleRun])).values()
    results[f"{da.coords['sample_rotation'].value} {da.coords['sample_rotation'].unit}"] = R.hist()

sc.plot(results, norm='log', vmin=1e-9)

In [ ]:
results = []
for path in estia_mcstas_example('Natural SiO2 on silicon'):
    w = wf.copy()
    w[Filename[SampleRun]] = path
    results.append(w.compute(ReducibleData[SampleRun]))

In [ ]:
wavelength_z_figure(results[3], wavelength_bins=400)

In [ ]:
wavelength_theta_figure(results, wavelength_bins=400, theta_bins=200, q_edges_to_display=[sc.scalar(0.016, unit='1/angstrom')])

In [ ]:
q_theta_figure(results, q_bins=300, theta_bins=200)